In [1]:
# initialize getLinks function to scrape the links
# to all organization home pages

from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re

random.seed(datetime.datetime.now())
def getLinks(articleUrl):
    html=urlopen("https://utexas.collegiatelink.net"+articleUrl)
    return bsObj.findAll("a", href=re.compile("^(/organization/)"))

In [2]:
# create orgLinks array to store the links to org homepage

orgLinks = []
for i in range(1,123):
    html = urlopen("https://utexas.collegiatelink.net/organizations?"+
               "SearchType=None&SelectedCategoryId=0&CurrentPage="+str(i))
    bsObj = BeautifulSoup(html.read(), "lxml")
    links = getLinks("/organization/")
    for link in links:
        if 'href' in link.attrs:
            orgLinks.append(link.attrs['href'])

In [6]:
# debug orgLinks

#for link in orgLinks:
#    print(link)
print(len(orgLinks))

1213


In [4]:
# Initialize webdriver and manually login to get the cookies

from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
import re

driver = webdriver.Chrome()
driver.get("https://utexas.collegiatelink.net/organization/igotsole")

In [7]:
# Get cookies after logging in and define getInformation function
# to scrape orgName, personName and emailAddress from each org page

cookies = driver.get_cookies()
for cookie in cookies:
    driver.add_cookie(cookie)

def getInformation(orgLink):
    driver.get("https://utexas.collegiatelink.net"+orgLink)
    # driver.get(orgLink)
    memberCard = None
    pageSource = driver.page_source
    pageSource = BeautifulSoup(pageSource, "lxml")
    orgName = pageSource.title
    orgName = orgName.get_text()
    orgName = orgName.replace(orgName[-10:],'')
    orgName = orgName.replace(orgName[:2],'')
    contact = pageSource.findAll("a", href=re.compile("^(/users/membercard/)"))
    for link in contact:
        if 'href' in link.attrs:
            memberCard = link.attrs['href']
        else:
            memberCard = None
    
    if memberCard == None:
        emailAddress = "None"
        personName = "None"
    else:
        driver.get("https://utexas.collegiatelink.net"+memberCard)
        emailPerson = driver.page_source
        emailPerson = BeautifulSoup(emailPerson, "lxml")
        personName = emailPerson.findAll("span", {"class":"fn"})
        for person in personName:
            personName = person.get_text()
        emailPerson = emailPerson.findAll("a", href=re.compile("^(mailto:)"))
        if emailPerson == []:
            emailAddress = "None"
        else:
            for email in emailPerson:
                if 'href' in email.attrs:
                    emailAddress = email.attrs['href']
                    emailAddress = emailAddress.replace(emailAddress[:7],'')
                
    time.sleep(3)
    return orgName, personName, emailAddress

In [8]:
# Initialize allOrgInfo array to store tuples containing org information
# use for loop to iterate through orgLinks array
allOrgInfo = []
for orgLink in orgLinks:
    orgInfo = getInformation(orgLink)
    allOrgInfo.append(orgInfo)

In [ ]:
# debug allOrgInfo to make sure all it contains all the necessary info 
for org in allOrgInfo:
    a, b, c = org
    print(a,b,c)

In [19]:
# save allOrgInfo into a Pandas Dataframe
import pandas as pd
orgDF = pd.DataFrame(allOrgInfo)

In [10]:
# print Dataframe to make sure all the information is present
print(orgDF)

                                                      0                   1  \
0                                            #IGOTSOLE    CHRISTIANA THOMAS   
1                                       (I)'s of Texas       MORGAN LAWLESS   
2                                    3-D Modeling Club         SAMIR HASSAM   
3                  4 Corners Christian Group at Austin             LITAN LI   
4     40th Annual Big XII Conference on Black Studen...                None   
5                                        Aaja Ve Texas         KRISHMA SHAH   
6                           ABC Residence Hall Council   UJWAL MADHAVAPEDDI   
7                                      Absolute Texxas       Summer WHEELER   
8                                    Acacia Fraternity    ZACKERY FERNANDEZ   
9                      ACADEMIC COUNSELORS ASSOCIATION           Kayla Ford   
10                         Access Christian Fellowship          JIA-YU CHIN   
11                                          ACLU at 

In [69]:
# convert the array containing all weblinks to student org pages
# into a Pandas DataFrame, so that we can combine it with orgDF
testLinks = pd.DataFrame(orgLinks)

In [70]:
# combine/concatenate the two DataFrames to join them by column
orgDF = pd.concat([orgDF, testLinks], axis=1)

In [71]:
# rename DataFrame columns to accurately reflect the data
orgDF.columns = ['Organization Name', 'Contact Name',
              'Contact Email', 'Organization Homepage']

In [73]:
# export DataFrame to a csv file
orgDF.to_csv("hornsLinkOrgData.csv")